# Load libraries

In [6]:
import os
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
from keras import *

# image processing
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import img_to_array

# build your own nets
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [8]:
train_image_files_path = "/keras2production/fruits/Training/"
valid_image_files_path = "/keras2production/fruits/Test/"

# Keras

https://keras.io/

- try out different fruits and combinations of fruits to classify

In [9]:
fruit_list = ["Apricot", "Avocado", "Banana", "Clementine", "Cocos", "Kiwi", "Lemon", "Limes", 
              "Mandarine", "Orange", "Peach", "Pineapple", "Plum", "Pomegranate", "Raspberry", "Strawberry"]

favorite_fruits = ["Banana", "Pineapple", "Pomegranate", "Kiwi", "Orange", "Lemon"]



- play around with different data augmentation settings

In [11]:
output_n = len(fruit_list)
size = 50
img_width = 50
img_height = 50
channels = 3
batch_size = 16

train_data_gen = ImageDataGenerator(
    rescale = 1 / 255,
    #rshear_range = 0.2,
    rotation_range = 40
    # optional data augmentation
    #rotation_range = 40,
    #rwidth_shift_range = 0.2,
    #rheight_shift_range = 0.2,
    #rshear_range = 0.2,
    #rzoom_range = 0.2,
    #rhorizontal_flip = TRUE,
    #rfill_mode = "nearest"
)

valid_data_gen = ImageDataGenerator(
    # validation data shouldn't be augmented
    rescale = 1 / 255
)

train_image_array_gen = train_data_gen.flow_from_directory(
    train_image_files_path,
    target_size = (img_width, img_height),
    class_mode = 'categorical',
    classes = fruit_list,
    color_mode = 'rgb', 
    batch_size = batch_size,
    seed = 42)

valid_image_array_gen = valid_data_gen.flow_from_directory(
    valid_image_files_path,
    target_size = (img_width, img_height),
    class_mode = 'categorical',
    classes = fruit_list,
    color_mode = 'rgb', 
    batch_size = batch_size,
    seed = 42)

Found 7709 images belonging to 16 classes.
Found 2428 images belonging to 16 classes.


- try out different layer combinations
- try out different hyperparameter settings
- train the model

In [12]:
model.add?

Object `model.add` not found.


In [13]:
epochs = 3
train_samples = train_image_array_gen.n
valid_samples = valid_image_array_gen.n
print(train_samples, valid_samples)

model = Sequential()

model.add(Conv2D(64, (3, 3), padding = "valid", input_shape = (img_width, img_height, channels)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)), padding = "same")
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.5))

model.add(Conv2D(16, (3, 3)), padding = "same")
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.5))

model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.125))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(output_n))
model.add(Activation('softmax'))

print(model.summary())

7709 2428


TypeError: add() got an unexpected keyword argument 'padding'

- download new images
- use them to predict based on your trained model

- Train another model with 20% validation split of the training data (Hint: ImageDataGenerator & flow_from_directory)
- Use `evaluate_generator` and `predict_generator` to measure performance of your model on the Test data folder